# Trabajo de Curso VC

### Autores
Kilian Armas Pérez

Pablo Segura López

In [ ]:
# pip install torch==1.10.0+cpu torchvision==0.11.0+cpu torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html
# pip install matplotlib
# pip install numpy
# pip install opencv-python
# pip install scipy
# pip install torchsummary
# pip install h5py
# pip install tqdm
# pip install yacs
# pip install numba
# pip install scikit-image
# pip install filterpy
# Añadir al path el ffmpeg, la carpeta bin

In [ ]:
#python gen_skes.py --video fortnite.mp4 --animation
#python reconstruction.py --video-path './data/video/fortnite.mp4'

# Convertir de npz a bvh
#python npz_to_bvh.py ./output/fortnite.npz ./output/fortnite.bvh

In [3]:
import subprocess
working_dir = "C:/Users/pablo/Desktop/Universidad/Cuarto/Visión por computador/Prácticas/Trabajo-VC/GAST-Net"
process = subprocess.Popen(["python",
                            "gen_skes.py",
                            "--video", "fortnite.mp4"
                            ], 
                            stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, cwd=working_dir)
for line in process.stdout:
    print(line,end="")

Loading YOLOv3 network.....
YOLOv3 network successfully loaded
Loading HRNet model ...
HRNet network successfully loaded
Generating 2D pose ...

  0%|          | 0/205 [00:00<?, ?it/s]No person detected!

  0%|          | 1/205 [00:00<02:03,  1.65it/s]No person detected!

  1%|          | 2/205 [00:01<02:04,  1.63it/s]No person detected!

  1%|▏         | 3/205 [00:01<02:02,  1.65it/s]No person detected!

100%|██████████| 205/205 [04:43<00:00,  1.38s/it]
Loading GAST-Net ...
GAST-Net successfully loaded
Generating 3D human pose ...
Saving 3D reconstruction...
Completing saving...


In [ ]:
import bpy
import numpy as np

# Ruta del archivo .npz con las posiciones de las articulaciones
npz_file_path = "/ruta/a/tu_archivo.npz"

# Nombre de los huesos en tu rig (modifica estos nombres según tu modelo)
bone_names = [
    "root", "spine", "chest", "neck", "head",  # Tronco
    "shoulder.L", "upper_arm.L", "forearm.L", "hand.L",  # Brazo izquierdo
    "shoulder.R", "upper_arm.R", "forearm.R", "hand.R",  # Brazo derecho
    "hip.L", "thigh.L", "shin.L", "foot.L",  # Pierna izquierda
    "hip.R", "thigh.R", "shin.R", "foot.R",  # Pierna derecha
]

# Carga los datos del archivo .npz
data = np.load(npz_file_path)
joints = data['joints']  # Suponemos que el archivo tiene una clave 'joints'
# Dimensiones esperadas: (num_frames, num_joints, 3)

num_frames, num_joints, _ = joints.shape

# Verifica que el número de huesos coincide con el número de articulaciones
if len(bone_names) != num_joints:
    raise ValueError("El número de huesos no coincide con el número de articulaciones en el archivo .npz")

# Selecciona el objeto con rigging
armature = bpy.data.objects["Armature"]  # Cambia "Armature" si tu rig tiene otro nombre
bpy.context.view_layer.objects.active = armature
bpy.ops.object.mode_set(mode='POSE')

# Crea animación fotograma por fotograma
for frame_idx in range(num_frames):
    bpy.context.scene.frame_set(frame_idx + 1)  # Configura el fotograma actual
    
    for joint_idx, bone_name in enumerate(bone_names):
        # Obtiene la posición de la articulación para este fotograma
        joint_pos = joints[frame_idx, joint_idx]
        
        # Encuentra el hueso correspondiente
        bone = armature.pose.bones.get(bone_name)
        if bone is None:
            print(f"Hueso {bone_name} no encontrado en el rig.")
            continue
        
        # Mueve el hueso a la posición deseada
        bone.location = joint_pos  # Ajusta la posición local
        
        # Insertar clave para la posición del hueso
        bone.keyframe_insert(data_path="location", frame=frame_idx + 1)

print("Animación completada.")
